# I. Cài đặt thư viện

In [1]:
import torch
from facenet_pytorch import MTCNN
import cv2
from datetime import datetime
import os

Để kiểm tra xem có gpu không, nếu có thì sau này tensor sẽ đưa lên đây tính toán cho nhanh

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


Kiểm tra xem đang dùng gpu loại gì

In [3]:
torch.cuda.get_device_name(0)

'GeForce GTX 1650'

# II.MTCNN

Gọi 1 object từ class MTCNN với các thông số sau
thresholds: Mức ngưỡng, những bounding box có tỷ lệ chính xác thấp hơn mức này sẽ bị xóa
keep_all: Xác định có detect và trả về toàn bộ ảnh trong bức hình không
margin: Kích thước của khối box
post_process: Có chuẩn hóa hay không


In [4]:
mtcnn = MTCNN(margin=20, keep_all = True, post_process=False, device= device)

In [5]:
count = 50 # Đếm số lượng ảnh
leap = 1  # bước nhảy: lấy 1 ảnh sau mỗi leap frame

# III. Load từ webcam
Cài đặt nguồn cấp dữ liệu là webcam với khung hình là 640x480

In [6]:
video_cap = cv2.VideoCapture(0)
video_cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
video_cap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)

True

Vị trí lưu ảnh sau khi lấy

In [7]:
img_path = './new_face'
label_face = input('Tên của bạn là gì: ')
face_path = os.path.join(img_path, label_face)
print(face_path)

./images_train\


Khoanh box cho từng frame

In [8]:
while video_cap.isOpened() and count:
    isSuccess, frame = video_cap.read()
    # xác định lại xem có đúng thu được 1 frame từ camera không
    if mtcnn(frame) is not None and leap%2 : # nếu có box nào đó tồn tại
        # Đặt tên theo định dạng 2021-07-03-22-39-2216.jpg
        path = str(face_path + '/{}.jpg'.format(str(datetime.now())[:-7].replace(":","-").replace(" ","-") + str(count)))
        # Nhận diện và lưu ảnh
        face_img = mtcnn(frame,save_path = path)
        count -= 1
    leap +=1
    cv2.imshow('Face_detection', frame)
    # Đặt nút tắt
    if cv2.waitKey(1)&0xFF == ord('q'):
        break
video_cap.release()
cv2.destroyAllWindows()